# Pythonでbcrypt イントロダクション

パスワード保管に適したハッシュ関数bcryptをPythonから利用する場合は、bcryptモジュール (<https://pypi.python.org/pypi/bcrypt/3.1.1>)を使います。

**インストール方法**:
- pip install bcrypt

**bcryptの特徴**

bcryptはBlowfish暗号ベースのハッシュ関数です。

MD5やSHAファミリなどのハッシュ関数とbcryptとの決定的な違いは、前者がfast hashであるのに対して、後者はslow hashであることです。

fast hashは高速であるため、大きなファイルから固定長のダイジェストを得るのには便利ですが、パスワード管理で使ってしまうとハッシュ値が漏れてしまった場合にオフライン攻撃によってクラックされやすいという問題があります。そのため、意図的に処理を遅くさせるストレッチ（ハッシュ化を繰り返す）という手法があります。

一方、bcryptはもとも演算を繰り返すように設計されたslow hashであり、また、高速なハードウェア実装を難しくする設計にもなっています。
> This system hashes passwords using a version of Bruce Schneier's Blowfish block cipher with modifications designed to raise the cost of off-line password cracking and frustrate fast hardware implementation. 

[引用元] http://www.mindrot.org/projects/py-bcrypt/

> Basically, computational power can be parallelized cheaply and easily, but memory cannot. This is the cornerstone of bcrypt and scrypt. Obviously, they can still be broken by sheer brute force, and you could just use hardware with integrated memory units to circumvent the problem, but it's much harder and much, much more expensive to do so.

[引用元] http://crypto.stackexchange.com/questions/3491/is-stretching-hash-several-times-basically-the-same-as-bcrypt

（正直、理解不足で上記の説明が正しいのか、今でも有効なのかよく分かってません。）

**bcryptの基本的な使い方**
 - bcrypt.gensalt(rounds=12, prefix=b'2b')  # ソルトを生成
 - bcrypt.hashpw(password, salt)  # パスワードをハッシュ化
 - bcrypt.checkpw(password, hashed_password)  # パスワードを検証

**bcrypt.gensalt()**の引数について

bcrypt.gensalt()は呼び出しごとに異なるソルトを生成します。また、呼び出し時に２つの引数を指定できます。  
1. rounds：ハッシュ値の計算を遅くするためのwork-factorと呼ばれるパラメータで、4から31の値が指定できます（default=12）。2^(rounds)回演算が繰り返されます。  
2. prefix：b'2a'かb'2b'を指定できます（default=b'2b'）。
> 現在は"2a"が主流になっているようです。

[引用元] https://yuskamiya.tumblr.com/post/100503173956

実験
---
以下では、bcrypt.gensalt()のwork-factorの値を変えたときの計算時間を見てみます。

**実行環境:**
- Windows 10 Pro 64bit Intel Core i5-4210U @ 1.70GHz, 8.0GB RAM

In [3]:
import bcrypt
password = 'password'.encode()
for work in range(5, 18):
    print('work: {:02d}'.format(work), end=' => ')
    %timeit -r1 bcrypt.hashpw(password, bcrypt.gensalt(rounds=work))

work: 05 => 100 loops, best of 1: 2.88 ms per loop
work: 06 => 100 loops, best of 1: 5.54 ms per loop
work: 07 => 100 loops, best of 1: 11 ms per loop
work: 08 => 10 loops, best of 1: 22.4 ms per loop
work: 09 => 10 loops, best of 1: 44.4 ms per loop
work: 10 => 10 loops, best of 1: 94.5 ms per loop
work: 11 => 1 loop, best of 1: 176 ms per loop
work: 12 => 1 loop, best of 1: 432 ms per loop
work: 13 => 1 loop, best of 1: 862 ms per loop
work: 14 => 1 loop, best of 1: 1.42 s per loop
work: 15 => 1 loop, best of 1: 2.95 s per loop
work: 16 => 1 loop, best of 1: 5.59 s per loop
work: 17 => 1 loop, best of 1: 11.4 s per loop


**実験結果**

当然ですが、work-factorを1増やすごとに計算時間が2倍になりました。  

まとめ
---
**1. Pythonからbcryptを利用する場合は、bcryptモジュールを使う**

**2. 適切なwork-factorを選ぶ**

work-factorに大きな値を設定することで、攻撃者のオフライン攻撃に対する時間稼ぎが可能ですが、これは通常運用時のパフォーマンスにも影響を与えます。セキュリティと利便性のトレードオフです。
> 望ましい時間（10、200msなど）内にサーバがパスワードをチェックする反復の回数を決定し、それを使用する。

[引用元] http://blog.f-secure.jp/archives/50564743.html

> I don’t believe that there is a “correct” work factor; it depends on how strong you want your hashes to be and how much computational power you want to reserve for the hashing process. 

[引用元] https://wildlyinaccurate.com/bcrypt-choosing-a-work-factor/